In [ ]:
%%configure -f
{
"conf": {
    "spark.dynamicAllocation.disableIfMinMaxNotSpecified.enabled": true,
    "spark.dynamicAllocation.enabled": true,
    "spark.dynamicAllocation.minExecutors": 2,
    "spark.dynamicAllocation.maxExecutors": 20
   }
}

In [ ]:
batch_id = ''
invoice_schema_applied_path = ''
invoice_cleaned_path = ''
statistics_path = ''
exchange_rates_file = ''
data_separator = ''
data_encoding = ''
drop_records_threshold = 0
local_currency_iso_code = ''

In [ ]:
import datetime
import csv
import pandas as pd
import os
from datetime import date
from calendar import monthrange
import time
from pyspark.sql.functions import col, year, month, dayofmonth, isnan, when, count, length, trim
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, TimestampType, FloatType

In [ ]:
# Initiate logging
import logging
from opencensus.ext.azure.log_exporter import AzureLogHandler
from opencensus.ext.azure.trace_exporter import AzureExporter
from opencensus.trace import config_integration
from opencensus.trace.samplers import AlwaysOnSampler
from opencensus.trace.tracer import Tracer

instrumentation_connection_string = mssparkutils.credentials.getSecretWithLS("keyvault", "AppInsightsConnectionString")
config_integration.trace_integrations(['logging'])

logger = logging.getLogger(__name__)
logger.addHandler(AzureLogHandler(connection_string=instrumentation_connection_string))
logger.setLevel(logging.INFO)

tracer = Tracer(
    exporter=AzureExporter(
        connection_string=instrumentation_connection_string
    ),
    sampler=AlwaysOnSampler()
)

# Spool parameters
run_time_parameters = {'custom_dimensions': {
    'batch_id': batch_id,
    'statistics_path': statistics_path,
    'data_encoding': data_encoding,
    'data_separator': data_separator,
    'invoice_schema_applied_path': invoice_schema_applied_path,
    'invoice_cleaned_path': invoice_cleaned_path,
    'exchange_rates_file': exchange_rates_file,
    'drop_records_threshold': drop_records_threshold,
    'local_currency_iso_code': local_currency_iso_code,
    'notebook_name': mssparkutils.runtime.context['notebookname']
} }
  
logger.info(f"{mssparkutils.runtime.context['notebookname']}: INITIALISED", extra=run_time_parameters)

In [ ]:
def deep_ls(path: str, max_depth=1):
    """
    List all files and folders in specified path and
    subfolders within maximum recursion depth.
    """

    # List all files in path
    li = mssparkutils.fs.ls(path)

    # Return all files
    for x in li:
        if x.size != 0:
            yield x

    # If the max_depth has not been reached, start
    # listing files and folders in subdirectories
    if max_depth > 1:
        for x in li:
            if x.size != 0:
                continue
            for y in deep_ls(x.path, max_depth - 1):
                yield y

    # If max_depth has been reached,
    # return the folders
    else:
        for x in li:
            if x.size == 0:
                yield x

In [ ]:
def get_total_nulls(df, colname) :
    if [dtype for name, dtype in df.dtypes if name == colname][0] not in ("timestamp", "date"):
        total_rows = df.where(col(colname).contains('None') | col(colname).contains('NULL') | (col(colname) == '' ) | col(colname).isNull() | isnan(colname)).count()
    else:
        total_rows = df.where(col(colname).contains('None') | col(colname).contains('NULL') | (col(colname) == '' ) | col(colname).isNull()).count()
    return total_rows

In [ ]:
def apply_cleansing_rules(fileName, fullFilePath, df_exchange_rate) :
    from pyspark.sql.functions import col, year, month, dayofmonth, isnan, when, count, length, trim
    from pyspark.sql.types import StringType, DateType, FloatType, IntegerType
    
    df = spark.read.csv(fullFilePath, sep=data_separator,inferSchema=True, header=True)

    # Change data types
    df = df.withColumn('issuer_type' , df['issuer_type'].cast(StringType()))
    df = df.withColumn('issuer_id' , df['issuer_id'].cast(StringType()))
    df = df.withColumn('activity_issuer' , df['activity_issuer'].cast(StringType()))
    df = df.withColumn('receiver_type' , df['receiver_type'].cast(StringType()))
    df = df.withColumn('receiver_id' , df['receiver_id'].cast(StringType()))
    df = df.withColumn('document_type' , df['document_type'].cast(StringType()))
    df = df.withColumn('document_id' , df['document_id'].cast(StringType()))
    df = df.withColumn('issued_date' , df['issued_date'].cast(TimestampType()))
    df = df.withColumn('sales_terms' , df['sales_terms'].cast(StringType()))
    df = df.withColumn('credit_term' , df['credit_term'].cast(IntegerType()))
    df = df.withColumn('currency' , df['currency'].cast(StringType()))
    df = df.withColumn('exchange_rate_r' , df['exchange_rate_r'].cast(FloatType()))
    df = df.withColumn('payment_method1' , df['payment_method1'].cast(StringType()))
    df = df.withColumn('payment_method2' , df['payment_method2'].cast(StringType()))
    df = df.withColumn('payment_method3' , df['payment_method3'].cast(StringType()))
    df = df.withColumn('payment_method4' , df['payment_method4'].cast(StringType()))
    df = df.withColumn('payment_method5' , df['payment_method5'].cast(StringType()))
    df = df.withColumn('payment_method5' , df['payment_method5'].cast(StringType()))
    df = df.withColumn('payment_method99' , df['payment_method99'].cast(StringType()))
    df = df.withColumn('total_taxable_services' , df['total_taxable_services'].cast(FloatType()))
    df = df.withColumn('total_non_taxable_services' , df['total_non_taxable_services'].cast(FloatType()))
    df = df.withColumn('total_taxable_goods' , df['total_taxable_goods'].cast(FloatType()))
    df = df.withColumn('total_non_taxable_goods' , df['total_non_taxable_goods'].cast(FloatType()))
    df = df.withColumn('total_taxable' , df['total_taxable'].cast(FloatType()))
    df = df.withColumn('total_non_taxable' , df['total_non_taxable'].cast(FloatType()))
    df = df.withColumn('total_sales' , df['total_sales'].cast(FloatType()))
    df = df.withColumn('total_discounts' , df['total_discounts'].cast(FloatType()))
    df = df.withColumn('total_voucher' , df['total_voucher'].cast(FloatType()))
    df = df.withColumn('total_tax' , df['total_tax'].cast(FloatType()))

    # Remove rows where null values are under the allowed threshold
    total_rows_in_dataset = df.count() 
    logger.info(f'Total rows in dataset to be cleaned: {total_rows_in_dataset}')

    #issuer_id.	Drop records if < 1%.Stop, review, fix if > 1%
    total_rows_null_issuer_id = get_total_nulls(df,"issuer_id")
    logger.info(f'Total rows with null issuer_id: {total_rows_null_issuer_id}')
    per_rows_null_issuer_id = total_rows_null_issuer_id / total_rows_in_dataset
    if (per_rows_null_issuer_id <= drop_records_threshold) :
        df.na.drop(subset=["issuer_id"])

    #issued_date. Drop records if < 1%.Stop, review, fix if > 1%
    total_rows_null_issued_date = get_total_nulls(df,"issued_date")
    per_rows_null_issued_date = total_rows_null_issued_date / total_rows_in_dataset
    if (per_rows_null_issuer_id <= drop_records_threshold) :
        df.na.drop(subset=["issued_date"])    

    #total_voucher. Drop records if < 1%.Stop, review, fix if > 1%
    total_rows_null_total_voucher = get_total_nulls(df,"total_voucher")
    per_rows_null_total_voucher = total_rows_null_total_voucher / total_rows_in_dataset
    if (per_rows_null_total_voucher <= drop_records_threshold) :
        df = df.filter(col("total_voucher").isNotNull() & ~isnan("total_voucher")) 

    #total_tax. Drop records if < 1%.Stop, review, fix if > 1% 
    total_rows_null_total_tax = get_total_nulls(df,"total_tax")
    per_rows_null_total_tax = total_rows_null_total_tax / total_rows_in_dataset
    if (per_rows_null_total_tax <= drop_records_threshold) :
        df = df.filter(col("total_tax").isNotNull() & ~isnan("total_tax")) 


    #total_voucher. Replace with 0.0 if Null. Replace iwth 0.0 if any alpha character
    df = df.fillna(value=0, subset=['total_voucher'])

    #total_tax. Replace with 0.0 if Null. Replace iwth 0.0 if any alpha character
    df = df.fillna(value=0, subset=['total_tax'])

    #document_id	String	8 to 100-character code that identifies the document number.	Set to “no_identified_document”
    df = df.fillna(value="no_identified_document", subset=['document_id'])

    #Currency	String	Currency code according to ISO 4217	Set to local currency
    df = df.fillna(value=local_currency_iso_code, subset=['currency'])

    #document_type	string	Alphanumeric code that identifies the type of electronic voucher: Example: I: Invoice, D: Debit Note, C: Credit Note, O: Order, G: Goods certificate, T: Tender Receipt, TC: Tender Contract, etc.	Set to “I” = to Invoice
    df = df.fillna(value='I', subset=['document_type'])

    #receiver_id	String	Tax ID associated with the taxpayer who received the e-Invoicing.	Set to no_identified_receiver
    df = df.fillna(value="no_identified_receiver", subset=['receiver_id'])

    #receiver_id. Set to no_identified_receiver if any value > 16 characters.
    df = df.withColumn("receiver_id", \
              when(length(df["receiver_id"]) > 16, 'no_identified_receiver').otherwise(df["receiver_id"]))

    #activity_issuer	String	Indicates the code of the economic activity to which the electronic receipt corresponds.	
    #Replace with activiy code "999999" if Null.
    #Replace with activiy code "999999" if lenght > 8 digits.
    #Replace with activiy code "999999" if length code < 4 digits
    
    df = df.fillna(value="999999", subset=['activity_issuer'])

    df = df.withColumn("activity_issuer", \
              when(length(df["activity_issuer"]) > 8, '999999').otherwise(df["activity_issuer"]))

    df = df.withColumn("activity_issuer", \
              when(length(df["activity_issuer"]) < 4, '999999').otherwise(df["activity_issuer"]))


    #total_taxable.	Set to 0 (currency)
    df = df.fillna(value=0, subset=['total_taxable'])

    #total_non_taxable.	Set to 0 (currency)
    df = df.fillna(value=0, subset=['total_non_taxable'])

    #total_sales. Set to 0 (currency)
    df = df.fillna(value=0, subset=['total_sales'])

    #total_discounts.	Set to 0 (currency)
    df = df.fillna(value=0, subset=['total_discounts'])

    #total_taxable_services.	Set to 0
    df = df.fillna(value=0, subset=['total_taxable_services'])

    #total_non_taxable_services.	Set to 0
    df = df.fillna(value=0, subset=['total_non_taxable_services'])

    #total_taxable_goods. Set to 0
    df = df.fillna(value=0, subset=['total_taxable_goods'])

    #total_non_taxable_goods. Set to 0
    df = df.fillna(value=0, subset=['total_non_taxable_goods'])  
  
    #issuer_type	Replace with "NT" if Null
    df = df.fillna(value="NT", subset=['issuer_type'])

    #receiver_type	Replace with "NT" if Null
    df = df.fillna(value="NT", subset=['receiver_type'])

    #sales_terms	Replace with "00" if Null
    df = df.fillna(value="00", subset=['sales_terms'])

    #credit_term	Replace with "00" if Null
    df = df.fillna(value="00", subset=['credit_term'])

    #exchange_rate_r. Replace with 1.0 if null
    df = df.fillna(value=1, subset=['exchange_rate_r'])

    #payment_method1	Replace with "00" if Null
    #payment_method1 Replace with "00" if any length > 2
    df = df.fillna(value="00", subset=['payment_method1'])
    df = df.withColumn("payment_method1", \
            when(length(df["payment_method1"]) > 2, '00').otherwise(df["payment_method1"]))

    #payment_method2	Replace with "00" if Null
    #payment_method2 Replace with "00" if any length > 2
    df = df.fillna(value="00", subset=['payment_method2'])
    df = df.withColumn("payment_method2", \
            when(length(df["payment_method2"]) > 2, '00').otherwise(df["payment_method2"]))

    #payment_method3	Replace with "00" if Null
    #payment_method3 Replace with "00" if any length > 2
    df = df.fillna(value="00", subset=['payment_method3'])
    df = df.withColumn("payment_method3", \
            when(length(df["payment_method3"]) > 2, '00').otherwise(df["payment_method3"]))

    #payment_method4	Replace with "00" if Null
    #payment_method4 Replace with "00" if any length > 2
    df = df.fillna(value="00", subset=['payment_method4'])
    df = df.withColumn("payment_method4", \
            when(length(df["payment_method4"]) > 2, '00').otherwise(df["payment_method4"]))

    #payment_method5	Replace with "00" if Null
    #payment_method5 Replace with "00" if any length > 2
    df = df.fillna(value="00", subset=['payment_method5'])
    df = df.withColumn("payment_method5", \
            when(length(df["payment_method5"]) > 2, '00').otherwise(df["payment_method5"]))

    #payment_method99	Replace with "00" if Null
    #payment_method99 Replace with "00" if any length > 2
    df = df.fillna(value="00", subset=['payment_method99'])
    df = df.withColumn("payment_method99", \
            when(length(df["payment_method99"]) > 2, '00').otherwise(df["payment_method99"]))

    # merge df and df_er so df has all the exchange rates
    
    df = df.join(df_exchange_rate, (df['currency'] == df_exchange_rate['currency_exchange_rate']) & (df['issued_date'] == df_exchange_rate['date_exchange_rate']), how='left')    

    #set the local currency to rows where the currency does not need to be converted

    df = df.fillna(value=local_currency_iso_code, subset=['currency_exchange_rate'])
    df = df.fillna(value=1, subset=['exchange_rate_value'])

    #remove columns no longer needed

    df = df.drop('date_exchange_rate') 


    # apply exchange rates and create new columns for freshly transformed columns
    monetary_cols = ['total_taxable_services',
                    'total_tax',
                    'total_non_taxable_services',
                    'total_non_taxable_goods',
                    'total_taxable_goods',
                    'total_taxable',
                    'total_non_taxable',
                    'total_sales',
                    'total_discounts',
                    'total_voucher']
    for col in monetary_cols:
        transformed_col = 'transformed_' + col
        df = df.withColumn(transformed_col,  df[col]*df['exchange_rate_value'])

    #remove columns no longer needed
    df = df.drop('currency_exchange_rate') 

    #return the cleansed dataset for this file
    return df
    

# Data cleaning 

In this code we read all the loaded files and we apply the cleaning rules, new columns are created using the exchange rate

In [ ]:
with tracer.span('Loading and parsing exchange rate file'):
    # read in exchange rate file
    logger.info('Reading in exchange rate dictionary')
    df_exchangerate = spark.read.csv(exchange_rates_file, sep=data_separator,inferSchema=True, header=True).toPandas()

    logger.info('Extracting exchange rate info')
    ignore_cols = ['ISO_CODE','DESCRIPTION']
    # only read in exchange rate values
    dates = [x for x in df_exchangerate.columns if x not in ignore_cols]
    exchange_rate_ll = []

    for row in df_exchangerate.iterrows():
        # ISO_CODE is basically just currency
        country = row[1]['ISO_CODE']
        for date in dates:
            # modify date format to match that of original df dates
            issued_date_parts = str(pd.to_datetime(date).date()).split('-')
            issued_date = issued_date_parts[0] + '-' + issued_date_parts[1] + '-' + issued_date_parts[2]
            # remove commas from exchange_rate num values (e.g. 1,000)
            exchange_rate = float(row[1][date])
            exchange_rate_ll.append([country,issued_date,exchange_rate])


    df_er = pd.DataFrame(exchange_rate_ll,columns=['currency_exchange_rate','date_exchange_rate','exchange_rate_value'])

    #Create PySpark DataFrame from Pandas
    df_exchange_rate =spark.createDataFrame(df_er) 

with tracer.span('Loading schema applied invoice files'):
    file_names = mssparkutils.fs.ls(invoice_schema_applied_path)

firstFile = True
for filename in file_names:  
    logger.info(f'Applying cleansing rules for invoice file: {filename.name}')
    with tracer.span('Applying cleansing rules for invoice file'):
        df_cleansed_file_data = apply_cleansing_rules(filename.name, filename.path, df_exchange_rate)
    
    if firstFile == True:
        df_all_cleansed_data = df_cleansed_file_data
        firstFile = False
    else:
        df_all_cleansed_data = df_all_cleansed_data.union(df_cleansed_file_data)


with tracer.span('Saving all cleansed data to ADLS'):
    df_all_cleansed_data.write.mode("overwrite").parquet(invoice_cleaned_path)